In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pandas as pd
import numpy as np
import pickle
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from pypinyin import pinyin, lazy_pinyin, Style
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
# autogluon保存路径
model_path = r"F:\cache_data\zone_ana\ky\modle\autogluon_type"
# 检查路径是否存在，否则便创建
if not os.path.exists(model_path):
    os.makedirs(model_path)
else:
    print("文件夹已存在")

文件夹已存在


In [10]:
dataset = pd.read_csv(r"F:\cache_data\zone_ana\ky\train_data\pca_soil_type_train_point.csv")

In [115]:
# # 筛选满足条件的记录
# pr = dataset[(dataset['MZMC'] == "河流冲积物") & (dataset['NEW_TZ'] == "中层砂质灰潮土")]

# # 如果筛选出的记录少于 5 个，我们将修改所有记录
# num_to_change = min(5, len(pr))

# if num_to_change > 0:
#     # 随机选择 num_to_change 个索引
#     random_indices = np.random.choice(pr.index, num_to_change, replace=False)
#     # 修改选中记录的 NEW_TZ 值
#     dataset.loc[random_indices, 'NEW_TZ'] = "厚层砂质灰潮土"

#     print(f"已随机修改 {num_to_change} 条记录的 NEW_TZ 值为 '厚层砂质灰潮土'")
# else:
#     print("没有找到满足条件的记录")

# # 验证修改结果
# modified_records = dataset[(dataset['MZMC'] == "河流冲积物") & (dataset['NEW_TZ'] == "厚层砂质灰潮土")]
# print(f"修改后，符合新条件的记录数：{len(modified_records)}")

In [11]:
[_ for _ in dataset.columns]

['point_id',
 'longitude',
 'latitude',
 'TL',
 'YL',
 'TS',
 'TZ',
 'label',
 'LON',
 'LAT',
 'DH',
 'NEW_TZ',
 'DL',
 'DZ',
 'PW',
 'LB',
 'analyticalhillshading',
 'aspect',
 'bio',
 'channelnetworkbaselevel',
 'channelnetworkdistance',
 'closeddepressions',
 'contrast',
 'convergenceindex',
 'correlation',
 'dem',
 'dissimilarity',
 'dl',
 'dz',
 'entropy',
 'etp2022_1',
 'etp2022_10',
 'etp2022_11',
 'etp2022_12',
 'etp2022_2',
 'etp2022_3',
 'etp2022_4',
 'etp2022_5',
 'etp2022_6',
 'etp2022_7',
 'etp2022_8',
 'etp2022_9',
 'etp2022_mean',
 'evi',
 'homogeneity',
 'lat',
 'lon',
 'lsfactor',
 'lswi',
 'mean',
 'mndwi',
 'mrttf',
 'mrvbf',
 'ndmi',
 'ndvi',
 'ndwi',
 'night2022',
 'pc1',
 'pc2',
 'plancurvature',
 'pre',
 'pre2022_1',
 'pre2022_10',
 'pre2022_11',
 'pre2022_12',
 'pre2022_2',
 'pre2022_3',
 'pre2022_4',
 'pre2022_5',
 'pre2022_6',
 'pre2022_7',
 'pre2022_8',
 'pre2022_9',
 'pre2022_mean',
 'profilecurvature',
 'relativeslopeposition',
 'savi',
 'secondmoment',
 's

In [12]:
dataset.shape

(6689, 107)

In [13]:
dataset["TZ_label"] = dataset.NEW_TZ.astype("category").cat.codes

In [14]:
dataset['dl'] = dataset['dl'].astype('category')
dataset['dz'] = dataset['dz'].astype('category')
dataset['slope_postion'] = dataset['slope_postion'].astype('category')
dataset['TZ_label'] = dataset['TZ_label'].astype('category')

In [15]:
result = dataset.groupby('TZ_label', observed=True)["NEW_TZ"].apply(lambda x: list(x.unique())).to_dict()
for one_type in result:
    result[one_type] = result[one_type][0]
print(result)

{0: '中层壤质中性紫色土', 1: '中层壤质灰潮土', 2: '中层壤质黄色石灰土', 3: '中层壤质黑色石灰土', 4: '中层暗泥质黄壤', 5: '中层泥质黄壤', 6: '中层泥质黄壤性土', 7: '中层灰泥质黄壤', 8: '中层砂泥质黄壤', 9: '中层砂质中性紫色土', 10: '中层砂质酸性紫色土', 11: '中层硅质黄壤', 12: '中层红泥质黄壤', 13: '中层黏质黄色石灰土', 14: '厚层壤质灰潮土', 15: '厚层壤质黄色石灰土', 16: '厚层暗泥质黄壤', 17: '厚层泥质黄壤', 18: '厚层泥质黄壤性土', 19: '厚层灰泥质黄壤', 20: '厚层砂泥质黄壤', 21: '厚层硅质黄壤', 22: '厚层红泥质黄壤', 23: '浅石灰泥田', 24: '渗紫泥田', 25: '潮泥田', 26: '潮泥砂田', 27: '石灰泥田', 28: '紫泥田', 29: '腐中层壤质中性紫色土', 30: '腐中层壤质黄色石灰土', 31: '腐中层壤质黑色石灰土', 32: '腐中层暗泥质黄壤', 33: '腐中层泥质黄壤', 34: '腐中层泥质黄棕壤', 35: '腐中层灰泥质黄壤', 36: '腐中层灰泥质黄棕壤', 37: '腐中层砂泥质黄壤', 38: '腐中层砂泥质黄棕壤', 39: '腐中层砂质中性紫色土', 40: '腐中层砂质灰潮土', 41: '腐中层砂质酸性紫色土', 42: '腐中层硅质黄壤', 43: '腐中层红泥质黄壤', 44: '腐中层黏质黄色石灰土', 45: '腐厚层壤质黄色石灰土', 46: '腐厚层壤质黑色石灰土', 47: '腐厚层暗泥质黄壤', 48: '腐厚层泥质黄壤', 49: '腐厚层泥质黄棕壤', 50: '腐厚层灰泥质黄壤', 51: '腐厚层灰泥质黄棕壤', 52: '腐厚层砂泥质黄壤', 53: '腐厚层砂质灰潮土', 54: '腐厚层砂质酸性紫色土', 55: '腐厚层硅质黄壤', 56: '腐厚层硅质黄壤土', 57: '腐厚层硅黄壤', 58: '腐厚层红泥质黄壤', 59: '腐薄层壤质黄色石灰土', 60: '腐薄层壤质黑色石灰土', 61: '腐薄层暗泥质黄壤', 62: '腐薄层泥质黄壤', 63: '腐薄层灰泥质黄壤', 64:

In [16]:
# 保存映射关系
with open(r'D:\worker_code\Terrain_Test\data\soil_dict_20241216_ky.json', 'w') as f:
    json.dump(result, f)

In [17]:
# 查看地类类别
tdlylx_df = pd.DataFrame(dataset['LB'].value_counts())
tdlylx_df

,count
LB,
自然土,3884
非自然土,2059
水稻土,746


In [18]:
# 查看地类类别
dl_df = pd.DataFrame(dataset['DL'].value_counts())
dl_df

,count
DL,
乔木林地,2585
旱地,1774
灌木林地,1178
水田,737
果园,201
其他林地,69
茶园,63
其他草地,49
设施农用地,15


In [19]:
# 查看母质类别
mz_df = pd.DataFrame(dataset['DZ'].value_counts())
mz_df

,count
DZ,
白云岩,2306
灰岩,1439
泥(页)岩,627
页岩,599
粘土岩,383
砂页岩,333
砂岩,310
紫色砂岩,243
变余凝灰岩,133


### 砂岩、变余砂岩土种

In [20]:
# 筛选砂岩类水稻土数据
sy_sdt_data = dataset[(dataset['LB']=='水稻土')
                      & ((dataset['DZ'] == '砂岩') | (dataset['DZ'] == '变余砂岩'))]
pd.unique(sy_sdt_data['NEW_TZ']),#pd.unique(sy_sdt_data['NEW_TS'])

(array(['黄白粉泥田', '黄浅白粉泥田', '轻漂白粉泥田'], dtype=object),)

In [21]:
# 筛砂岩选非自然土数据  
sy_fzrt_data = dataset[(dataset['LB']=='非自然土') 
                       & ((dataset['DZ'] == '砂岩') | (dataset['DZ'] == '变余砂岩'))]
pd.unique(sy_fzrt_data['NEW_TZ']),#pd.unique(sy_fzrt_data['NEW_TS'])

(array(['中层硅质黄壤', '薄层硅质黄壤', '厚层硅质黄壤'], dtype=object),)

In [22]:
# 筛选自然土数据
sy_zrt_data = dataset[(dataset['LB']=='自然土')
                   & ((dataset['DZ'] == '砂岩') | (dataset['DZ'] == '变余砂岩'))]
pd.unique(sy_zrt_data['NEW_TZ']),#pd.unique(sy_zrt_data['NEW_TS'])

(array(['腐中层硅质黄壤', '腐薄层硅质黄壤', '腐厚层硅质黄壤', '腐厚层硅质黄壤土', '腐厚层硅黄壤'],
       dtype=object),)

### 碳酸岩土种

In [23]:
# 筛选碳酸岩水稻土数据
tsy_sdt_data = dataset[(dataset['LB']=='水稻土')
                      & ((dataset['DZ'] == '白云岩')|(dataset['DZ'] == '灰岩') | (dataset['DZ'] == '变余凝灰岩'))]
pd.unique(tsy_sdt_data['NEW_TZ']),#pd.unique(tsy_sdt_data['NEW_TS']),

(array(['石灰泥田', '青石灰泥田', '黄灰泥田', '浅石灰泥田', '轻漂石灰泥田', '黄石灰泥田'], dtype=object),)

In [24]:
# 筛选碳酸岩选非自然土数据
tsy_fzrt_data = dataset[(dataset['LB']=='非自然土')
                       & ((dataset['DZ'] == '白云岩')|(dataset['DZ'] == '灰岩') | (dataset['DZ'] == '变余凝灰岩'))]

pd.unique(tsy_fzrt_data['NEW_TZ']),#pd.unique(tsy_fzrt_data['NEW_TS']),

(array(['中层灰泥质黄壤', '薄层灰泥质黄壤', '厚层灰泥质黄壤', '中层壤质黄色石灰土', '薄层壤质黄色石灰土',
        '薄层壤质黑色石灰土', '厚层壤质黄色石灰土', '中层壤质黑色石灰土', '中层黏质黄色石灰土'], dtype=object),)

In [25]:
# 筛选碳酸岩自然土数据

tsy_zrt_data = dataset[(dataset['LB']=='自然土')
                       & ((dataset['DZ'] == '白云岩')|(dataset['DZ'] == '灰岩') | (dataset['DZ'] == '变余凝灰岩'))]
pd.unique(tsy_zrt_data['NEW_TZ']),#pd.unique(tsy_zrt_data['NEW_TS']),

(array(['腐中层灰泥质黄壤', '腐中层壤质黄色石灰土', '腐厚层灰泥质黄壤', '腐薄层壤质黄色石灰土', '腐薄层灰泥质黄壤',
        '腐薄层壤质黑色石灰土', '腐厚层壤质黄色石灰土', '腐中层壤质黑色石灰土', '腐中层黏质黄色石灰土',
        '腐厚层壤质黑色石灰土', '腐中层灰泥质黄棕壤', '腐厚层灰泥质黄棕壤'], dtype=object),)

### 第四系红粘土

In [26]:
# 筛选第四系红粘土水稻土数据
hnt_sdt_data = dataset[(dataset['LB']=='水稻土')
                       & ((dataset['DZ'] == '第四纪粘土'))]
pd.unique(hnt_sdt_data['NEW_TZ']),#pd.unique(hnt_sdt_data['NEW_TS']),

(array(['黄红泥田', '黄渗红泥田'], dtype=object),)

In [27]:
# 筛选第四系红粘土非自然土数据
hnt_fzrt_data = dataset[(dataset['LB']=='非自然土')
                       & ((dataset['DZ'] == '第四纪粘土'))]
pd.unique(hnt_fzrt_data['NEW_TZ']),#pd.unique(hnt_fzrt_data['NEW_TS']),

(array(['中层红泥质黄壤', '厚层红泥质黄壤'], dtype=object),)

In [28]:
# 筛选第四系红粘土自然土数据
hnt_zrt_data = dataset[(dataset['LB']=='自然土')
                       & ((dataset['DZ'] == '第四纪粘土'))]
pd.unique(hnt_zrt_data['NEW_TZ']),#pd.unique(hnt_zrt_data['NEW_TS']),

(array(['腐中层红泥质黄壤', '腐厚层红泥质黄壤'], dtype=object),)

### 泥(页)岩

In [29]:
# 筛选泥(页)岩水稻土数据
nyy_sdt_data = dataset[(dataset['LB']=='水稻土')
                       & ((dataset['DZ'] == '泥(页)岩') | (dataset['DZ'] == '页岩') | (dataset['DZ'] == '粘土岩') | (dataset['DZ'] == '板岩'))]
pd.unique(nyy_sdt_data['NEW_TZ']),#pd.unique(nyy_sdt_data['NEW_TS']),

(array(['黄鳝泥田', '黄浅鳝泥田', '黄青鳝泥田', '轻漂鳝泥田'], dtype=object),)

In [30]:
# 筛选泥(页)岩非自然土数据
nyy_fzrt_data = dataset[(dataset['LB']=='非自然土')
                       & ((dataset['DZ'] == '泥(页)岩') | (dataset['DZ'] == '页岩') | (dataset['DZ'] == '粘土岩') | (dataset['DZ'] == '板岩'))]
pd.unique(nyy_fzrt_data['NEW_TZ']),#pd.unique(nyy_fzrt_data['NEW_TS']),

(array(['中层泥质黄壤', '厚层泥质黄壤', '薄层泥质黄壤', '薄层泥质黄壤性土', '中层泥质黄壤性土', '厚层泥质黄壤性土'],
       dtype=object),)

In [31]:
# 筛选泥(页)岩自然土数据    
nyy_zrt_data = dataset[(dataset['LB']=='自然土')
                       & ((dataset['DZ'] == '泥(页)岩') | (dataset['DZ'] == '页岩') | (dataset['DZ'] == '粘土岩') | (dataset['DZ'] == '板岩'))]
pd.unique(nyy_zrt_data['NEW_TZ']),#pd.unique(nyy_zrt_data['NEW_TS']),

(array(['腐中层泥质黄壤', '腐厚层泥质黄壤', '腐薄层泥质黄壤', '腐中层泥质黄棕壤', '腐厚层泥质黄棕壤'],
       dtype=object),)

### 紫红色砂页岩	

In [32]:
# 筛选紫红色砂页岩水稻土数据
zhsyy_sdt_data = dataset[(dataset['LB']=='水稻土')
                       & ((dataset['DZ'] == '紫色砂岩'))]
pd.unique(zhsyy_sdt_data['NEW_TZ']),#pd.unique(zhsyy_sdt_data['NEW_TS']),

(array(['紫泥田', '渗紫泥田'], dtype=object),)

In [33]:
# 筛选紫红色砂页岩非自然土数据
zhsyy_fzrt_data = dataset[(dataset['LB']=='非自然土')
                       & ((dataset['DZ'] == '紫色砂岩'))]
pd.unique(zhsyy_fzrt_data['NEW_TZ']),#pd.unique(zhsyy_fzrt_data['NEW_TS']),

(array(['中层砂质酸性紫色土', '中层壤质中性紫色土', '中层砂质中性紫色土'], dtype=object),)

In [34]:
# 筛选紫红色砂页岩自然土数据
zhsyy_zrt_data = dataset[(dataset['LB']=='自然土')
                       & ((dataset['DZ'] == '紫色砂岩'))]
pd.unique(zhsyy_zrt_data['NEW_TZ']),#pd.unique(zhsyy_zrt_data['NEW_TS']),

(array(['腐中层壤质中性紫色土', '腐中层砂质酸性紫色土', '腐中层砂质中性紫色土', '腐薄层砂质酸性紫色土',
        '腐厚层砂质酸性紫色土'], dtype=object),)

### 河流冲积物

In [35]:
# 筛选河流冲积物水稻土数据
hlcjw_sdt_data = dataset[(dataset['LB']=='水稻土')
                       & (dataset['DZ'] == '河流冲积物')]
pd.unique(hlcjw_sdt_data['NEW_TZ']),#pd.unique(hlcjw_sdt_data['NEW_TS']),

(array(['潮泥田', '潮泥砂田'], dtype=object),)

In [36]:
# 筛选河流冲积物非自然土数据
hlcjw_fzrt_data = dataset[(dataset['LB']=='非自然土')
                       & (dataset['DZ'] == '河流冲积物')]
pd.unique(hlcjw_fzrt_data['NEW_TZ']),#pd.unique(hlcjw_fzrt_data['NEW_TS']),

(array(['厚层壤质灰潮土', '中层壤质灰潮土'], dtype=object),)

In [37]:
# 筛选河流冲积物自然土数据
hlcjw_zrt_data = dataset[(dataset['LB']=='自然土')
                       & (dataset['DZ'] == '河流冲积物')]
pd.unique(hlcjw_zrt_data['NEW_TZ']),#pd.unique(hlcjw_zrt_data['NEW_TS']),


(array(['腐中层砂质灰潮土', '腐厚层砂质灰潮土'], dtype=object),)

### 砂页岩

In [38]:
# 筛选砂页岩水稻土数据
syy_sdt_data = dataset[(dataset['LB']=='水稻土')
                       & ((dataset['DZ'] == '砂页岩') | (dataset['DZ'] == '砾岩'))]
pd.unique(syy_sdt_data['NEW_TZ']),#pd.unique(syy_sdt_data['NEW_TS']),

(array(['黄砂泥田', '黄浅砂泥田', '黄青砂泥田'], dtype=object),)

In [39]:
# 筛选砂页岩非自然土数据
syy_fzrt_data = dataset[(dataset['LB']=='非自然土')
                       & ((dataset['DZ'] == '砂页岩') | (dataset['DZ'] == '砾岩'))]
pd.unique(syy_fzrt_data['NEW_TZ']),#pd.unique(syy_fzrt_data['NEW_TS']),

(array(['中层砂泥质黄壤', '厚层砂泥质黄壤', '薄层砂泥质黄壤'], dtype=object),)

In [40]:
# 筛选砂页岩物自然土数据
syy_zrt_data = dataset[(dataset['LB']=='自然土')
                       & ((dataset['DZ'] == '砂页岩') | (dataset['DZ'] == '砾岩'))]
pd.unique(syy_zrt_data['NEW_TZ']),#pd.unique(syy_zrt_data['NEW_TS']),


(array(['腐中层砂泥质黄壤', '腐厚层砂泥质黄壤', '腐薄层砂泥质黄壤', '腐中层砂泥质黄棕壤'], dtype=object),)

# 玄武岩

In [41]:
# 筛选玄武岩水稻土数据
xw_sdt_data = dataset[(dataset['LB']=='水稻土')
                       & ((dataset['DZ'] == '玄武岩'))]
pd.unique(xw_sdt_data['NEW_TZ']),#pd.unique(xw_sdt_data['NEW_TS']),

(array(['黄渗暗泥田', '黄暗泥田'], dtype=object),)

In [42]:
# 筛选玄武岩自然土数据
xw_zrt_data = dataset[(dataset['LB']=='自然土')
                       & ((dataset['DZ'] == '玄武岩'))]
pd.unique(xw_zrt_data['NEW_TZ']),#pd.unique(xw_zrt_data['NEW_TS']),


(array(['腐中层暗泥质黄壤', '腐厚层暗泥质黄壤', '腐薄层暗泥质黄壤'], dtype=object),)

In [43]:
# 筛选玄武岩非自然土数据
xw_fzrt_data = dataset[(dataset['LB']=='非自然土')
                       & ((dataset['DZ'] == '玄武岩'))]
pd.unique(xw_fzrt_data['NEW_TZ']),#pd.unique(xw_fzrt_data['NEW_TS']),



(array(['中层暗泥质黄壤', '厚层暗泥质黄壤'], dtype=object),)

In [44]:
columns_List = [_ for _ in dataset.columns]
columns_List

['point_id',
 'longitude',
 'latitude',
 'TL',
 'YL',
 'TS',
 'TZ',
 'label',
 'LON',
 'LAT',
 'DH',
 'NEW_TZ',
 'DL',
 'DZ',
 'PW',
 'LB',
 'analyticalhillshading',
 'aspect',
 'bio',
 'channelnetworkbaselevel',
 'channelnetworkdistance',
 'closeddepressions',
 'contrast',
 'convergenceindex',
 'correlation',
 'dem',
 'dissimilarity',
 'dl',
 'dz',
 'entropy',
 'etp2022_1',
 'etp2022_10',
 'etp2022_11',
 'etp2022_12',
 'etp2022_2',
 'etp2022_3',
 'etp2022_4',
 'etp2022_5',
 'etp2022_6',
 'etp2022_7',
 'etp2022_8',
 'etp2022_9',
 'etp2022_mean',
 'evi',
 'homogeneity',
 'lat',
 'lon',
 'lsfactor',
 'lswi',
 'mean',
 'mndwi',
 'mrttf',
 'mrvbf',
 'ndmi',
 'ndvi',
 'ndwi',
 'night2022',
 'pc1',
 'pc2',
 'plancurvature',
 'pre',
 'pre2022_1',
 'pre2022_10',
 'pre2022_11',
 'pre2022_12',
 'pre2022_2',
 'pre2022_3',
 'pre2022_4',
 'pre2022_5',
 'pre2022_6',
 'pre2022_7',
 'pre2022_8',
 'pre2022_9',
 'pre2022_mean',
 'profilecurvature',
 'relativeslopeposition',
 'savi',
 'secondmoment',
 's

In [45]:
features_use = ['analyticalhillshading','aspect','channelnetworkbaselevel','channelnetworkdistance',
 'convergenceindex','dem','dz','dl','etp2022_1','etp2022_10','etp2022_11','etp2022_12','etp2022_2','etp2022_3',
 'etp2022_4','etp2022_5','etp2022_6','etp2022_7','etp2022_8','etp2022_9','etp2022_mean','evi','lat','lon','lswi',
 'lsfactor','mndwi','mrttf','mrvbf','ndmi','ndvi','ndwi','night2022','pc1','pc2','plancurvature',
 'pre2022_1','pre2022_10','pre2022_11','pre2022_12','pre2022_2','pre2022_3','pre2022_4','pre2022_5','pre2022_6',
 'pre2022_7','pre2022_8','pre2022_9','pre2022_mean','profilecurvature','relativeslopeposition','savi','slope',
 'slope_postion','tmp2022_1','tmp2022_10','tmp2022_11','tmp2022_12',
 'tmp2022_2','tmp2022_3','tmp2022_4','tmp2022_5','tmp2022_6','tmp2022_7','tmp2022_8','tmp2022_9','tmp2022_mean',
 'topographicwetnessindex','totalcatchmentarea','valleydepth','vari']

In [46]:

sdt_features = features_use
fzrt_features = features_use
zrt_features = features_use
target = "TZ_label"

### 砂岩数据集

In [47]:
# 获取sdt数据集
sy_sdt_data = sy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
sy_fzrt_data = sy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
sy_zrt_data = sy_zrt_data[zrt_features+[target]]

### 碳酸岩数据集

In [48]:
# 获取sdt数据集
tsy_sdt_data = tsy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
tsy_fzrt_data = tsy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
tsy_zrt_data = tsy_zrt_data[zrt_features+[target]]

### 第四系红粘土数据集

In [49]:
# 获取sdt数据集
hnt_sdt_data = hnt_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
hnt_fzrt_data = hnt_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
hnt_zrt_data = hnt_zrt_data[zrt_features+[target]]

### 泥(页)岩数据集

In [50]:
# 获取sdt数据集
nyy_sdt_data = nyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
nyy_fzrt_data = nyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
nyy_zrt_data = nyy_zrt_data[zrt_features+[target]]

### 紫红色砂页岩数据集

In [51]:
# 获取sdt数据集
zhsyy_sdt_data = zhsyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
zhsyy_fzrt_data = zhsyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
zhsyy_zrt_data = zhsyy_zrt_data[zrt_features+[target]]

### 河流冲积物数据集

In [52]:
# 获取sdt数据集
hlcjw_sdt_data = hlcjw_sdt_data[sdt_features+[target]]
# 获取非自然土
hlcjw_fzrt_data = hlcjw_fzrt_data[fzrt_features+[target]]
# 获取自然土
hlcjw_zrt_data = hlcjw_zrt_data[zrt_features+[target]]

### 砂页岩数据集

In [53]:
# 获取sdt数据集
syy_sdt_data = syy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
syy_fzrt_data = syy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
syy_zrt_data = syy_zrt_data[zrt_features+[target]]

# 玄武岩数据集

In [54]:
# 获取水稻土数据集
xw_sdt_data = xw_sdt_data[sdt_features+[target]]
# 获取峨嵋山玄武岩数据集
xw_zrt_data = xw_zrt_data[zrt_features+[target]]
# 获取峨嵋山玄武岩非自然土数据集
xw_fzrt_data = xw_fzrt_data[fzrt_features+[target]]

In [55]:
# 查看各个数据集的大小
print('砂岩',sy_sdt_data.shape,sy_fzrt_data.shape,sy_zrt_data.shape)
print('碳酸岩',tsy_sdt_data.shape,tsy_fzrt_data.shape,tsy_zrt_data.shape)
print('第四系红粘土',hnt_sdt_data.shape,hnt_fzrt_data.shape,hnt_zrt_data.shape)
print('泥页岩',nyy_sdt_data.shape,nyy_fzrt_data.shape,nyy_zrt_data.shape)
print('紫红色砂页岩',zhsyy_sdt_data.shape,zhsyy_fzrt_data.shape,zhsyy_zrt_data.shape)
print('河流冲积物',hlcjw_sdt_data.shape,hlcjw_fzrt_data.shape,hlcjw_zrt_data.shape)
print('砂页岩',syy_sdt_data.shape,syy_fzrt_data.shape,syy_zrt_data.shape)
print('峨嵋山玄武岩',xw_sdt_data.shape,xw_zrt_data.shape,xw_fzrt_data.shape)

# 计算总数
total_data = pd.concat([sy_sdt_data,sy_fzrt_data,sy_zrt_data,tsy_sdt_data,tsy_fzrt_data,tsy_zrt_data,hnt_sdt_data,hnt_fzrt_data,hnt_zrt_data,nyy_sdt_data,nyy_fzrt_data,nyy_zrt_data,zhsyy_sdt_data,zhsyy_fzrt_data,zhsyy_zrt_data,hlcjw_sdt_data,hlcjw_fzrt_data,hlcjw_zrt_data,syy_sdt_data,syy_fzrt_data,syy_zrt_data,xw_sdt_data,xw_zrt_data,xw_fzrt_data])
total_data.shape,dataset.shape[0]


砂岩 (76, 72) (100, 72) (205, 72)
碳酸岩 (343, 72) (1242, 72) (2293, 72)
第四系红粘土 (11, 72) (14, 72) (12, 72)
泥页岩 (196, 72) (477, 72) (942, 72)
紫红色砂页岩 (17, 72) (35, 72) (191, 72)
河流冲积物 (26, 72) (30, 72) (23, 72)
砂页岩 (58, 72) (121, 72) (159, 72)
峨嵋山玄武岩 (19, 72) (59, 72) (40, 72)


((6689, 72), 6689)

In [56]:
# 确定标签
label = "TZ_label"
eval_metric = 'f1_weighted'
problem_type = 'multiclass'
# # 10折交叉验证
# cv_num = 10
# 初始化模型和超参数
hyperparameters={
	'NN_TORCH': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}]
}


### 模型训练

In [57]:
def train_model(train_data,label,eval_metric,hyperparameters,problem_type,model_master,model_branch,model_root):
    # 训练model
    temp_master_path =os.path.join(model_root,model_master)
    temp_branch_path = os.path.join(temp_master_path,model_branch)
    # 检查路径是否存在，否则便创建
    if not os.path.exists(temp_branch_path):
        os.makedirs(temp_branch_path)
    else:
        print("文件夹已存在")
    # 执行训练
    train_predictor = TabularPredictor(label=label,path=temp_branch_path,eval_metric=eval_metric,problem_type=problem_type).fit(train_data,hyperparameters=hyperparameters)
    train_predictor.fit_summary()
    return train_predictor.model_best


### 模型训练

In [58]:
# 砂岩
sy_sdt_predictor = train_model(sy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='sdt',model_root=model_path) 
sy_fzrt_predictor = train_model(sy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='fzrt',model_root=model_path) 
sy_zrt_predictor = train_model(sy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='zrt',model_root=model_path) 
# 碳酸岩
tsy_sdt_predictor = train_model(tsy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='sdt',model_root=model_path) 
tsy_fzrt_predictor = train_model(tsy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='fzrt',model_root=model_path) 
tsy_zrt_predictor = train_model(tsy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='zrt',model_root=model_path) 
# 第四系红粘土
hnt_sdt_predictor = train_model(hnt_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='sdt',model_root=model_path) 
hnt_fzrt_predictor = train_model(hnt_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='fzrt',model_root=model_path) 
hnt_zrt_predictor = train_model(hnt_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='zrt',model_root=model_path) 
# 泥页岩
nyy_sdt_predictor = train_model(nyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='sdt',model_root=model_path) 
nyy_fzrt_predictor = train_model(nyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='fzrt',model_root=model_path) 
nyy_zrt_predictor = train_model(nyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='zrt',model_root=model_path) 
# 紫红色砂页岩
zhsyy_sdt_predictor = train_model(zhsyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='sdt',model_root=model_path) 
zhsyy_fzrt_predictor = train_model(zhsyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='fzrt',model_root=model_path) 
zhsyy_zrt_predictor = train_model(zhsyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='zrt',model_root=model_path) 
# 河流冲积物
hlcjw_sdt_predictor = train_model(hlcjw_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='sdt',model_root=model_path)
hlcjw_fzrt_predictor = train_model(hlcjw_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='fzrt',model_root=model_path)
hlcjw_zrt_predictor = train_model(hlcjw_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='zrt',model_root=model_path)
# 砂页岩
syy_sdt_predictor = train_model(syy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='sdt',model_root=model_path) 
syy_fzrt_predictor = train_model(syy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='fzrt',model_root=model_path) 
syy_zrt_predictor = train_model(syy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='zrt',model_root=model_path) 
# 峨嵋山玄武岩
xw_sdt_predictor = train_model(xw_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='xw',model_branch='sdt',model_root=model_path) 
xw_zrt_predictor = train_model(xw_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='xw',model_branch='zrt',model_root=model_path) 
xw_fzrt_predictor = train_model(xw_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='xw',model_branch='fzrt',model_root=model_path) 

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\ky\modle\autogluon_type\sy\sdt"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          20
Memory Avail:       42.89 GB / 63.81 GB (67.2%)
Disk Space Avail:   286.51 GB / 

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       NeuralNetTorch   0.642857  f1_weighted       0.009091  1.322595                0.009091           1.322595            1       True          6
1  WeightedEnsemble_L2   0.642857  f1_weighted       0.013214  1.414037                0.004123           0.091442            2       True          7
2       ExtraTreesGini   0.611111  f1_weighted       0.024468  0.359447                0.024468           0.359447            1       True          4
3       ExtraTreesEntr   0.611111  f1_weighted       0.028930  0.363950                0.028930           0.363950            1       True          5
4     RandomForestGini   0.611111  f1_weighted       0.029641  0.356213                0.029641           0.356213            1       True          2
5     RandomForestEntr   0.611111  f1_

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\ky\modle\autogluon_type\sy\fzrt"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          20
Memory Avail:       42.67 GB / 63.81 GB (66.9%)
Disk Space Avail:   286.50 GB /

Plot summary of models saved to file: F:\cache_data\zone_ana\ky\modle\autogluon_type\sy\sdtSummaryOfModels.html
*** End of fit() summary ***


	0.4073	 = Validation score   (f1_weighted)
	0.45s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.3912	 = Validation score   (f1_weighted)
	0.36s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.3445	 = Validation score   (f1_weighted)
	0.38s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.3867	 = Validation score   (f1_weighted)
	0.35s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.385	 = Validation score   (f1_weighted)
	0.36s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.5011	 = Validation score   (f1_weighted)
	0.4s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetTorch': 0.875, 'ExtraTreesGini': 0.125}
	0.5211	 = Validation score   (f1_weighted)
	0.09s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGl

...

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       ExtraTreesGini   1.000000  f1_weighted       0.024948  0.374025                0.024948           0.374025            1       True          3
1     RandomForestEntr   1.000000  f1_weighted       0.028866  0.386088                0.028866           0.386088            1       True          2
2       ExtraTreesEntr   1.000000  f1_weighted       0.032455  0.367698                0.032455           0.367698            1       True          4
3     RandomForestGini   1.000000  f1_weighted       0.032816  0.376179                0.032816           0.376179            1       True          1
4  WeightedEnsemble_L2   1.000000  f1_weighted       0.036951  0.451444                0.004135           0.075265            2       True          6
5      NeuralNetFastAI   0.333333  f1_

No improvement since epoch 3: early stopping
	0.6429	 = Validation score   (f1_weighted)
	0.27s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.6429	 = Validation score   (f1_weighted)
	0.39s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.6429	 = Validation score   (f1_weighted)
	0.38s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.6429	 = Validation score   (f1_weighted)
	0.39s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.6429	 = Validation score   (f1_weighted)
	0.38s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.6429	 = Validation score   (f1_weighted)
	0.2s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'ExtraTreesEntr': 1.0}
	0.6429	 = Validation score   (f1_weighted)
	0.09s	 = Training   runtime
	0.0s	 = Validat

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      NeuralNetFastAI   0.642857  f1_weighted       0.004014  0.272080                0.004014           0.272080            1       True          1
1       NeuralNetTorch   0.642857  f1_weighted       0.012533  0.202199                0.012533           0.202199            1       True          6
2       ExtraTreesGini   0.642857  f1_weighted       0.027076  0.387155                0.027076           0.387155            1       True          4
3     RandomForestGini   0.642857  f1_weighted       0.028547  0.386343                0.028547           0.386343            1       True          2
4     RandomForestEntr   0.642857  f1_weighted       0.028731  0.381423                0.028731           0.381423            1       True          3
5  WeightedEnsemble_L2   0.642857  f1_

	0.8167	 = Validation score   (f1_weighted)
	0.35s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8167	 = Validation score   (f1_weighted)
	0.4s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.8167	 = Validation score   (f1_weighted)
	0.36s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.8167	 = Validation score   (f1_weighted)
	0.37s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
No improvement since epoch 6: early stopping
	0.891	 = Validation score   (f1_weighted)
	0.28s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.8167	 = Validation score   (f1_weighted)
	0.21s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetFastAI': 1.0}
	0.891	 = Validation score   (f1_weighted)
	0.09s	 = Training   runtime
	0.0s	 = Validati

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      NeuralNetFastAI   0.891026  f1_weighted       0.007521  0.277569                0.007521           0.277569            1       True          5
1  WeightedEnsemble_L2   0.891026  f1_weighted       0.008560  0.368524                0.001038           0.090955            2       True          7
2       NeuralNetTorch   0.816667  f1_weighted       0.006941  0.205652                0.006941           0.205652            1       True          6
3     RandomForestGini   0.816667  f1_weighted       0.029089  0.352210                0.029089           0.352210            1       True          1
4       ExtraTreesGini   0.816667  f1_weighted       0.029212  0.355592                0.029212           0.355592            1       True          3
5     RandomForestEntr   0.816667  f1_